In [1]:
import tensorflow as tf
import numpy as np

In [2]:
tf.__version__

'1.2.0'

In [3]:
from seq2seq.rnn_seq2seq import create_seq2seq_model, create_seq2seq_experiment_fn

In [4]:
tf.reset_default_graph()

In [5]:
! rm -r ./logs

rm: cannot remove './logs': No such file or directory


In [6]:
cell_num = 1
lm_regularization = True

cell = "LSTMCell"
num_layers = 2
num_units = 16
bidirectional = False
attention = True
residual_connections = True
residual_dense = True

vocab_size = 10
emb_size = 5

training_mode = "scheduled_sampling_embedding"
scheduled_sampling_probability = 0.5

inference_mode = "beam"
beam_width = 3

In [7]:
def random_sequences(length_from, length_to,
                     vocab_lower, vocab_upper,
                     batch_size):
    """ Generates batches of random integer sequences,
        sequence length in [length_from, length_to],
        vocabulary in [vocab_lower, vocab_upper]
    """
    if length_from > length_to:
        raise ValueError('length_from > length_to')

    def random_length():
        if length_from == length_to:
            return length_from
        return np.random.randint(length_from, length_to + 1)
    
    while True:
        yield [
            np.random.randint(low=vocab_lower,
                              high=vocab_upper,
                              size=random_length()).tolist()
            for _ in range(batch_size)]

In [8]:
# from tensorflow.contrib.learn.python.learn.learn_io.generator_io import generator_input_fn
from seq2seq.input.generator_io import generator_input_fn

In [9]:
def data_generator_py():
    def generator():
        data_gen = random_sequences(2, 9, 2, 9, 1)
        for _ in range(1000):
            data = next(data_gen)
            
            data = np.array(data[0], dtype=np.int32)
            data_len = len(data)
            data_len = np.array(data_len, dtype=np.int32)

            yield {
                "inputs": data,
                "inputs_length": data_len,
                "targets": data,
                "targets_length": data_len
            }
    return generator

In [10]:
dg_train_input_fn = generator_input_fn(
    x=data_generator_py(), 
    target_key=["targets", "targets_length"], 
    batch_size=16, shuffle=False, num_epochs=None, 
    queue_capacity=128, num_threads=2, 
    pad_data=True)
dg_test_input_fn = generator_input_fn(
    x=data_generator_py(), 
    target_key=["targets", "targets_length"], 
    batch_size=16, shuffle=False, num_epochs=None, 
    queue_capacity=128, num_threads=1, 
    pad_data=True)

In [11]:
experiment_fn = create_seq2seq_experiment_fn(
    dg_train_input_fn, dg_test_input_fn, 
    train_steps=int(1e3), eval_steps=int(1e2), min_eval_frequency=int(1e2))

In [12]:
gpu_option = 0.5  # Yeap, Estimator can use memory limitations
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_option)

run_config = tf.contrib.learn.RunConfig(
    session_config=tf.ConfigProto(gpu_options=gpu_options),
    model_dir="./logs")

hparams = tf.contrib.training.HParams(
    cell_num=cell_num,
    lm_regularization=lm_regularization,
    vocab_size=vocab_size, embedding_size=emb_size,
    cell=cell, num_layers=num_layers, num_units=num_units,
    bidirectional=bidirectional, attention=attention,
    residual_connections=residual_connections, 
    residual_dense=residual_dense,
    training_mode=training_mode,
    learning_rate=1e-4,
    lr_decay_steps=100000,
    lr_decay_koef=0.99,
    gradient_clip=10.0,
    scheduled_sampling_probability=scheduled_sampling_probability,
    inference_mode=inference_mode,
    beam_width=beam_width)

In [13]:
tf.logging.set_verbosity(tf.logging.ERROR) # cause we don't want to log anything
tf.contrib.learn.learn_runner.run(
    experiment_fn=experiment_fn,
    run_config=run_config,
    schedule="train_and_evaluate",
    hparams=hparams)

> /home/scitator/Documents/git/TF-seq2seq/seq2seq/rnn_seq2seq.py(86)__init__()
-> if mode == tf.estimator.ModeKeys.TRAIN:
(Pdb) l
 81  	            encoder_inputs_length=self.encoder.inputs_length,
 82  	            mode=mode,
 83  	            **decoder_args)
 84  	        import pdb; pdb.set_trace()
 85  	
 86  ->	        if mode == tf.estimator.ModeKeys.TRAIN:
 87  	            build_model_optimization(
 88  	                self.encoder, encoder_optimization_args, self.decoder.loss + self.encoder.loss)
 89  	            build_model_optimization(
 90  	                self.decoder, decoder_optimization_args)
 91  	            if same_embeddings:
(Pdb) from pprint import pprint; pprint([(v.name, v.get_shape()) for v in tf.trainable_variables()])
[('embeddings/embeddings/embedding_matrix:0',
  TensorShape([Dimension(10), Dimension(5)])),
 ('DynamicRnnEncoder/Encoder/rnn/extended_multi_rnn_cell/input_transform/weights:0',
  TensorShape([Dimension(5), Dimension(16)])),
 ('DynamicRnnEnco

/home/scitator/.conda/envs/python3/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


> /home/scitator/Documents/git/TF-seq2seq/seq2seq/rnn_seq2seq.py(86)__init__()
-> if mode == tf.estimator.ModeKeys.TRAIN:
(Pdb) from pprint import pprint; pprint([(v.name, v.get_shape()) for v in tf.trainable_variables()])
[('embeddings/embeddings/embedding_matrix:0',
  TensorShape([Dimension(10), Dimension(5)])),
 ('DynamicRnnEncoder/Encoder/rnn/extended_multi_rnn_cell/input_transform/weights:0',
  TensorShape([Dimension(5), Dimension(16)])),
 ('DynamicRnnEncoder/Encoder/rnn/extended_multi_rnn_cell/input_transform/biases:0',
  TensorShape([Dimension(16)])),
 ('DynamicRnnEncoder/Encoder/rnn/extended_multi_rnn_cell/cell_0/lstm_cell/kernel:0',
  TensorShape([Dimension(32), Dimension(64)])),
 ('DynamicRnnEncoder/Encoder/rnn/extended_multi_rnn_cell/cell_0/lstm_cell/bias:0',
  TensorShape([Dimension(64)])),
 ('DynamicRnnEncoder/Encoder/rnn/extended_multi_rnn_cell/cell_1/lstm_cell/kernel:0',
  TensorShape([Dimension(32), Dimension(64)])),
 ('DynamicRnnEncoder/Encoder/rnn/extended_multi_rnn_c

({'global_step': 1002, 'loss': 2.5092196}, [])

In [14]:
dg_pred_input_fn = generator_input_fn(
    x=data_generator_py(), 
    target_key=["targets", "targets_length"], 
    batch_size=1, shuffle=False, num_epochs=None, 
    queue_capacity=128, num_threads=1, 
    pad_data=True)

In [15]:
model = create_seq2seq_model(config=run_config, hparams=hparams)

In [16]:
predictions = model.predict(dg_pred_input_fn)

In [17]:
pred = next(predictions)

> /home/scitator/Documents/git/TF-seq2seq/seq2seq/rnn_seq2seq.py(86)__init__()
-> if mode == tf.estimator.ModeKeys.TRAIN:
(Pdb) from pprint import pprint; pprint([(v.name, v.get_shape()) for v in tf.trainable_variables()])
[('embeddings/embeddings/embedding_matrix:0',
  TensorShape([Dimension(10), Dimension(5)])),
 ('DynamicRnnEncoder/Encoder/rnn/extended_multi_rnn_cell/input_transform/weights:0',
  TensorShape([Dimension(5), Dimension(16)])),
 ('DynamicRnnEncoder/Encoder/rnn/extended_multi_rnn_cell/input_transform/biases:0',
  TensorShape([Dimension(16)])),
 ('DynamicRnnEncoder/Encoder/rnn/extended_multi_rnn_cell/cell_0/lstm_cell/kernel:0',
  TensorShape([Dimension(32), Dimension(64)])),
 ('DynamicRnnEncoder/Encoder/rnn/extended_multi_rnn_cell/cell_0/lstm_cell/bias:0',
  TensorShape([Dimension(64)])),
 ('DynamicRnnEncoder/Encoder/rnn/extended_multi_rnn_cell/cell_1/lstm_cell/kernel:0',
  TensorShape([Dimension(32), Dimension(64)])),
 ('DynamicRnnEncoder/Encoder/rnn/extended_multi_rnn_c

NotFoundError: Key DynamicRnnDecoder/Decoder/decoder/output_layer/bias not found in checkpoint
	 [[Node: save/RestoreV2_5 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_arg_save/Const_0_0, save/RestoreV2_5/tensor_names, save/RestoreV2_5/shape_and_slices)]]

Caused by op 'save/RestoreV2_5', defined at:
  File "/home/scitator/.conda/envs/python3/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/scitator/.conda/envs/python3/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/scitator/.conda/envs/python3/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/scitator/.conda/envs/python3/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/scitator/.conda/envs/python3/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 474, in start
    iolo

In [ ]:
pred

In [ ]:
from pprint import pprint; pprint([(v.name, v.get_shape()) for v in tf.trainable_variables()])

In [ ]:
# at the prediction 
[('embeddings/embeddings/embedding_matrix:0',
  TensorShape([Dimension(10), Dimension(5)])),
 ('DynamicRnnEncoder/Encoder/rnn/extended_multi_rnn_cell/input_transform/weights:0',
  TensorShape([Dimension(5), Dimension(16)])),
 ('DynamicRnnEncoder/Encoder/rnn/extended_multi_rnn_cell/input_transform/biases:0',
  TensorShape([Dimension(16)])),
 ('DynamicRnnEncoder/Encoder/rnn/extended_multi_rnn_cell/cell_0/lstm_cell/kernel:0',
  TensorShape([Dimension(32), Dimension(64)])),
 ('DynamicRnnEncoder/Encoder/rnn/extended_multi_rnn_cell/cell_0/lstm_cell/bias:0',
  TensorShape([Dimension(64)])),
 ('DynamicRnnEncoder/Encoder/rnn/extended_multi_rnn_cell/cell_1/lstm_cell/kernel:0',
  TensorShape([Dimension(32), Dimension(64)])),
 ('DynamicRnnEncoder/Encoder/rnn/extended_multi_rnn_cell/cell_1/lstm_cell/bias:0',
  TensorShape([Dimension(64)]))]